In [ ]:
# Check if GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA device count: {torch.cuda.device_count()}")


In [ ]:
# Clone the repository
!git clone https://github.com/YOUR_USERNAME/Comp430_Project.git
%cd Comp430_Project


In [ ]:
# Install dependencies
%pip install -r requirements.txt


In [ ]:
# Add repo path to Python path for imports
import sys
import os
import json
import time
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set project root
project_root = os.getcwd()
sys.path.insert(0, project_root)
print(f"Project root: {project_root}")

# Create output directory
output_dir = Path(project_root) / "experiments" / "out" / "colab_runs"
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output directory: {output_dir}")


In [ ]:
# List available configuration files
config_files = list(Path(project_root).glob("configs/*.yaml"))
print(f"Found {len(config_files)} configuration files:")
for config_file in sorted(config_files):
    print(f"- {config_file.name}")


In [ ]:
# Define our experiment suite
experiment_configs = [
    # MNIST with different client counts (vanilla DP)
    {"config": "mnist_clients3_vanilla_dp.yaml", "run_id": "mnist_clients3_vanilla", "desc": "MNIST with 3 clients (vanilla DP)"},
    {"config": "mnist_clients5_vanilla_dp.yaml", "run_id": "mnist_clients5_vanilla", "desc": "MNIST with 5 clients (vanilla DP)"},
    {"config": "mnist_clients10_vanilla_dp.yaml", "run_id": "mnist_clients10_vanilla", "desc": "MNIST with 10 clients (vanilla DP)"},
    {"config": "mnist_clients20_vanilla_dp.yaml", "run_id": "mnist_clients20_vanilla", "desc": "MNIST with 20 clients (vanilla DP)"},
    
    # MNIST with adaptive DP
    {"config": "mnist_clients5_adaptive_dp.yaml", "run_id": "mnist_clients5_adaptive", "desc": "MNIST with 5 clients (adaptive DP)"},
    
    # Split strategies
    {"config": "mnist_cut_layer_fl.yaml", "run_id": "mnist_fl_sim", "desc": "MNIST simulating Federated Learning (cut at last layer)"},
    {"config": "mnist_cut_layer_central.yaml", "run_id": "mnist_central_sim", "desc": "MNIST simulating Centralized Learning (cut at first layer)"},
    
    # Breast Cancer Wisconsin dataset
    {"config": "bcw_clients5_vanilla_dp.yaml", "run_id": "bcw_clients5_vanilla", "desc": "BCW with 5 clients (vanilla DP)"},
    {"config": "bcw_clients5_adaptive_dp.yaml", "run_id": "bcw_clients5_adaptive", "desc": "BCW with 5 clients (adaptive DP)"}
]

print(f"Defined {len(experiment_configs)} experiments to run")


In [ ]:
# Function to run an experiment with progress tracking
def run_experiment(config_file, run_id, description):
    print(f"\n{'='*80}")
    print(f"Starting experiment: {description}")
    print(f"Config file: {config_file}")
    print(f"Run ID: {run_id}")
    print(f"{'='*80}\n")
    
    # Create timestamp for this run
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    run_id_with_timestamp = f"{run_id}_{timestamp}"
    
    # Run the experiment
    start_time = time.time()
    cmd = f"python experiments/train_secure_sfl.py --config configs/{config_file} --run_id {run_id_with_timestamp}"
    print(f"Command: {cmd}")
    !{cmd}
    end_time = time.time()
    
    # Calculate total runtime
    total_runtime = end_time - start_time
    print(f"\nExperiment completed in {total_runtime:.2f} seconds ({total_runtime/60:.2f} minutes)")
    
    # Return experiment metadata
    return {
        "config": config_file,
        "run_id": run_id_with_timestamp,
        "description": description,
        "start_time": start_time,
        "end_time": end_time,
        "total_runtime": total_runtime,
        "metrics_path": f"experiments/out/{run_id_with_timestamp}/metrics.json"
    }


In [ ]:
# Run all experiments
all_results = []

for i, experiment in enumerate(experiment_configs):
    print(f"\nRunning experiment {i+1}/{len(experiment_configs)}")
    result = run_experiment(
        experiment['config'],
        experiment['run_id'],
        experiment['desc']
    )
    all_results.append(result)
    
# Save experiment summary
summary_file = output_dir / "experiment_summary.json"
with open(summary_file, 'w') as f:
    json.dump(all_results, f, indent=2)
    
print(f"\nAll {len(experiment_configs)} experiments completed")
print(f"Summary saved to {summary_file}")


In [ ]:
# Load experiment summary
with open(summary_file, 'r') as f:
    experiment_summary = json.load(f)
    
# Create a DataFrame for easier analysis
results_data = []

for exp in experiment_summary:
    try:
        # Load metrics for this experiment
        metrics_path = Path(project_root) / exp['metrics_path']
        if not metrics_path.exists():
            print(f"Warning: No metrics found for {exp['run_id']}")
            continue
            
        with open(metrics_path, 'r') as f:
            metrics = json.load(f)
        
        # Extract key metrics
        results_data.append({
            'run_id': exp['run_id'],
            'description': exp['description'],
            'config': exp['config'],
            'runtime_minutes': exp['total_runtime'] / 60,
            'final_accuracy': metrics.get('final_test_acc', None),
            'final_epsilon': metrics.get('privacy', {}).get('final_epsilon', None),
            'dataset': metrics.get('config', {}).get('dataset', '').lower(),
            'num_clients': metrics.get('config', {}).get('num_clients', 0),
            'batch_size': metrics.get('config', {}).get('batch_size', 0),
            'cut_layer': metrics.get('config', {}).get('cut_layer', 0),
            'rounds': metrics.get('config', {}).get('num_rounds', 0),
            'dp_mode': metrics.get('config', {}).get('dp_noise', {}).get('mode', 'unknown')
        })
    except Exception as e:
        print(f"Error processing {exp['run_id']}: {e}")
        
# Create DataFrame
results_df = pd.DataFrame(results_data)
results_df


In [ ]:
# Plot accuracy vs privacy budget (epsilon)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=results_df, x='final_epsilon', y='final_accuracy', 
                hue='dataset', style='dp_mode', size='num_clients', 
                sizes=(50, 200), alpha=0.7)

plt.title('Accuracy vs. Privacy Trade-off')
plt.xlabel('Privacy Budget (ε)')
plt.ylabel('Test Accuracy')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig(output_dir / 'accuracy_vs_privacy.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Filter for client scaling experiments (MNIST with vanilla DP)
client_scaling_df = results_df[results_df['config'].str.contains('mnist_clients') & 
                              results_df['config'].str.contains('vanilla')].sort_values('num_clients')

# Create comparison plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot accuracy vs number of clients
sns.barplot(x='num_clients', y='final_accuracy', data=client_scaling_df, ax=ax1)
ax1.set_title('Accuracy vs. Number of Clients')
ax1.set_xlabel('Number of Clients')
ax1.set_ylabel('Test Accuracy')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# Plot runtime vs number of clients
sns.barplot(x='num_clients', y='runtime_minutes', data=client_scaling_df, ax=ax2)
ax2.set_title('Runtime vs. Number of Clients')
ax2.set_xlabel('Number of Clients')
ax2.set_ylabel('Runtime (minutes)')
ax2.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(output_dir / 'client_scaling_analysis.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Filter for split strategy experiments
split_strategy_df = results_df[results_df['config'].str.contains('mnist') & 
                              (results_df['config'].str.contains('cut_layer') | 
                               results_df['run_id'].str.contains('clients5_vanilla'))]

# Add a human-readable split strategy column
def get_split_strategy(row):
    if 'central' in row['run_id']:
        return 'Centralized'
    elif 'fl_sim' in row['run_id']:
        return 'Federated Learning'
    else:
        return f"Split Learning (cut={row['cut_layer']})"

split_strategy_df['split_strategy'] = split_strategy_df.apply(get_split_strategy, axis=1)

# Plot comparison
plt.figure(figsize=(12, 6))

# Create grouped bar plot
metrics = ['final_accuracy', 'final_epsilon', 'runtime_minutes']
metric_names = ['Test Accuracy', 'Privacy Budget (ε)', 'Runtime (minutes)']

# Normalize values for better visualization
normalized_df = split_strategy_df.copy()
for metric in metrics:
    max_val = normalized_df[metric].max()
    if max_val > 0:  # Avoid division by zero
        normalized_df[f'{metric}_norm'] = normalized_df[metric] / max_val

# Create plot for each metric
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (metric, name) in enumerate(zip(metrics, metric_names)):
    sns.barplot(x='split_strategy', y=metric, data=split_strategy_df, ax=axes[i])
    axes[i].set_title(f'{name} by Split Strategy')
    axes[i].set_xlabel('Split Strategy')
    axes[i].set_ylabel(name)
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)
    axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig(output_dir / 'split_strategy_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Filter for DP comparison experiments
dp_comparison_df = results_df[
    ((results_df['config'].str.contains('mnist_clients5_vanilla') | 
      results_df['config'].str.contains('mnist_clients5_adaptive')) |
     (results_df['config'].str.contains('bcw_clients5_vanilla') | 
      results_df['config'].str.contains('bcw_clients5_adaptive')))
]

# Add a dataset-dp column for grouping
dp_comparison_df['dataset_dp'] = dp_comparison_df.apply(
    lambda row: f"{row['dataset']}-{'adaptive' if 'adaptive' in row['run_id'] else 'vanilla'}", 
    axis=1
)

# Plot accuracy evolution over time
plt.figure(figsize=(12, 8))

# For each experiment, load history and plot accuracy evolution
for _, exp in dp_comparison_df.iterrows():
    metrics_path = Path(project_root) / exp['metrics_path'].replace(exp['run_id'], exp['run_id'])
    try:
        with open(metrics_path, 'r') as f:
            metrics = json.load(f)
        
        if 'history' in metrics:
            rounds = metrics['history']['round']
            accuracy = metrics['history']['accuracy']
            
            # Plot line
            plt.plot(rounds, accuracy, marker='o', alpha=0.7, 
                     label=f"{exp['dataset'].upper()} - {'Adaptive' if 'adaptive' in exp['run_id'] else 'Vanilla'} DP")
    except Exception as e:
        print(f"Error processing history for {exp['run_id']}: {e}")

plt.title('Accuracy Evolution: Adaptive vs. Vanilla DP')
plt.xlabel('Training Round')
plt.ylabel('Test Accuracy')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.savefig(output_dir / 'dp_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Plot privacy budget evolution for a few selected experiments
selected_experiments = [
    'mnist_clients5_vanilla',
    'mnist_clients5_adaptive',
    'mnist_fl_sim',
    'bcw_clients5_vanilla'
]

plt.figure(figsize=(12, 6))

for exp_type in selected_experiments:
    matching_exps = [exp for exp in experiment_summary if exp['run_id'].startswith(exp_type)]
    
    if matching_exps:
        exp = matching_exps[0]
        metrics_path = Path(project_root) / exp['metrics_path']
        
        try:
            with open(metrics_path, 'r') as f:
                metrics = json.load(f)
            
            if 'history' in metrics:
                rounds = metrics['history']['round']
                privacy_epsilon = metrics['history']['privacy_epsilon']
                
                plt.plot(rounds, privacy_epsilon, marker='o', alpha=0.7, label=exp_type)
        except Exception as e:
            print(f"Error processing privacy history for {exp_type}: {e}")

plt.title('Privacy Budget Evolution')
plt.xlabel('Training Round')
plt.ylabel('Privacy Budget (ε)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.savefig(output_dir / 'privacy_budget_evolution.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Create a final summary table
summary_table = results_df[['description', 'final_accuracy', 'final_epsilon', 'runtime_minutes', 
                           'num_clients', 'cut_layer', 'dp_mode']]

# Rename columns for better readability
summary_table = summary_table.rename(columns={
    'description': 'Experiment',
    'final_accuracy': 'Accuracy', 
    'final_epsilon': 'Privacy Budget (ε)', 
    'runtime_minutes': 'Runtime (min)',
    'num_clients': 'Clients',
    'cut_layer': 'Cut Layer',
    'dp_mode': 'DP Mode'
})

# Format numbers
summary_table['Accuracy'] = summary_table['Accuracy'].map(lambda x: f"{x:.4f}" if pd.notnull(x) else "N/A")
summary_table['Privacy Budget (ε)'] = summary_table['Privacy Budget (ε)'].map(lambda x: f"{x:.4f}" if pd.notnull(x) else "N/A")
summary_table['Runtime (min)'] = summary_table['Runtime (min)'].map(lambda x: f"{x:.2f}" if pd.notnull(x) else "N/A")

# Display the table
summary_table


In [ ]:
# Save summary as HTML and CSV
summary_table.to_html(output_dir / 'experiment_summary_table.html')
summary_table.to_csv(output_dir / 'experiment_summary_table.csv')

print(f"Summary table saved to {output_dir}")
